In [7]:
import pandas as pd
from pathlib import Path

In [ ]:
try:
    project_root = Path.cwd().parent
    clean_data_path = project_root / "data" / "processed" / "credit_card_clients_clean.csv"
    df = pd.read_csv(clean_data_path)
except FileNotFoundError:
    print(f"Error: No se encontró el archivo: {clean_data_path}")

In [6]:
df['utilization_sept'] = df['bill_amt_sept'] / (df['limit_bal'] + 1)
df['utilization_aug'] = df['bill_amt_aug'] / (df['limit_bal'] + 1)


KeyError: 'bill_amt_sept'